In [1]:
import sys
import pandas as pd
import numpy as np
import urllib
import sqlalchemy

In [2]:
#Import shared functions
sys.path.append('..\..')
from IPM_Shared_Code_public.Python.utils import get_config

In [3]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
gis_server = config['srv']['server']
pip_server = config['srv']['pip']
gis = config['db']['parksgis']
pip = config['db']['pip']

In [4]:
con_string = 'Driver={' + driver + '};Server=' + pip_server +';Database=' + pip + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
pip_engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [5]:
con_string = 'Driver={' + driver + '};Server=' + gis_server +';Database=' + gis + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
gis_engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

## Set the prop id lookup field dictionary

In [6]:
propid_lookup = {'property_evw': 'gispropnum', 
                 'playground_evw': 'omppropid', 
                 'zone_evw': 'omppropid', 
                 'unmapped_gisallsites_evw': 'omppropid',
                 'schoolyard_to_playground_evw': 'gispropnum',
                 'greenstreet_evw': 'omppropid',
                 'golfcourse_evw': 'omppropid',
                 'restrictivedeclarationsite_evw': 'gispropnum',
                 'structure_evw': 'omppropid'}

In [7]:
field_lookup = {'property_evw': ['gispropnum', 'department', 'signname', 'location', 'acres', 'jurisdiction', 
                                 'typecategory', 'featurestatus', 'gisobjid'], 
                 'playground_evw': ['parentid', 'omppropid', 'department', 'signname', 'location', 'acres', 
                                    'jurisdiction', 'typecategory', 'featurestatus', 'gisobjid'],  
                 'zone_evw': ['parentid', 'omppropid', 'department', 'sitename', 'location', 'acres', 
                              'jurisdiction', 'featurestatus', 'gisobjid'], 
                 'unmapped_gisallsites_evw': ['gispropnum', 'omppropid', 'department', 'name', 'location', 
                                              'acres', 'jurisdiction'],
                 'schoolyard_to_playground_evw': ['gispropnum', 'department', 'signname', 'location', 'acres', 
                                                  'jurisdiction', 'featurestatus'], 
                 'greenstreet_evw': ['gispropnum', 'omppropid', 'department', 'sitename', 'location', 'acres', 
                                    'jurisdiction', 'featurestatus', 'gisobjid'], 
                 'golfcourse_evw': ['gispropnum', 'omppropid', 'department', 'name', 'location', 'acres', 
                                    'jurisdiction', 'featurestatus', 'gisobjid'],
                 'restrictivedeclarationsite_evw': ['gispropnum', 'department', 'signname', 'location', 'acres', 
                                                    'jurisdiction', 'featurestatus'],
                 'structure_evw': ['gispropnum', 'omppropid', 'department', 'description', 'location', 
                                   'jurisdiction', 'featurestatus', 'gisobjid']}

In [8]:
sourcefc_lookup = {'property_evw': 'Property', 
                   'playground_evw': 'Playground', 
                   'zone_evw': 'Zone', 
                   'unmapped_gisallsites_evw': 'Unmapped',
                   'schoolyard_to_playground_evw': 'Schoolyard To Playground',
                   'greenstreet_evw': 'Greenstreet',
                   'golfcourse_evw': 'GolfCourse',
                   'restrictivedeclarationsite_evw': 'RestrictiveDeclarationSite',
                   'structure_evw': 'Structure'}

## Original GIS Data

In [10]:
#Define the tables that will be queried and interacted with
gis_tables = ['property_evw', 'playground_evw', 'zone_evw', 'unmapped_gisallsites_evw', 
              'schoolyard_to_playground_evw', 'greenstreet_evw', 'golfcourse_evw', 
              'restrictivedeclarationsite_evw', 'structure_evw']

In [11]:
#Create the list of SQL Queries
gis_sql_list = ["select {}, {} as [prop id], '{}' as sourcefc from parksgis.dpr.{}"
                .format(' ,'.join(field_lookup[t]), propid_lookup[t], sourcefc_lookup[t], t) 
                        for t in gis_tables]

In [12]:
gis_sql_list

["select gispropnum ,department ,signname ,location ,acres ,jurisdiction ,typecategory ,featurestatus ,gisobjid, gispropnum as [prop id], 'Property' as sourcefc from parksgis.dpr.property_evw",
 "select parentid ,omppropid ,department ,signname ,location ,acres ,jurisdiction ,typecategory ,featurestatus ,gisobjid, omppropid as [prop id], 'Playground' as sourcefc from parksgis.dpr.playground_evw",
 "select parentid ,omppropid ,department ,sitename ,location ,acres ,jurisdiction ,featurestatus ,gisobjid, omppropid as [prop id], 'Zone' as sourcefc from parksgis.dpr.zone_evw",
 "select gispropnum ,omppropid ,department ,name ,location ,acres ,jurisdiction, omppropid as [prop id], 'Unmapped' as sourcefc from parksgis.dpr.unmapped_gisallsites_evw",
 "select gispropnum ,department ,signname ,location ,acres ,jurisdiction ,featurestatus, gispropnum as [prop id], 'Schoolyard To Playground' as sourcefc from parksgis.dpr.schoolyard_to_playground_evw",
 "select gispropnum ,omppropid ,department ,s

In [14]:
#Create a list of dataframes with the original data
gis_df_list = [pd.read_sql(con = gis_engine, sql = q) for q in gis_sql_list]

## Original PIP Data

In [15]:
#Define the tables that will be queried and interacted with
pip_tables = ['tbl_ref_allsites', 'tbl_ref_allsites_nosync', 'tbl_pip_allsites', 
              'vw_pip_compatible_inspected_sites', 'allsites']

In [16]:
#Create the list of SQL Queries
pip_sql_list = ['select * from accessnewpip.dbo.'+ t for t in pip_tables]

In [17]:
#Create a list of dataframes with the original dat
pip_df_list = [pd.read_sql(con = pip_engine, sql = q) for q in pip_sql_list]

In [18]:
pip_df_list[0]

,PropNum,Prop ID,Boro,AMPSDistrict,Prop Name,Site Name,Prop Location,Site Location,jurisdiction,typecategory,acres,gisobjid,sourcefc,created_date,row_hash,gis_deleted,shape
0,B001,B001,B,01,American Playground,American Playground,Franklin St. bet. Milton St. and Noble St.,Franklin St. bet. Milton St. and Noble St.,DPR,Playground,0.896,100004272.0,Property,2021-04-20 21:25:30.530,b'\x9f\xba\x91\xda1\xd9\xae\x1eT\x91\xfe\xd7P[...,False,"b""\xd7\x08\x00\x00\x01\x04\x05\x00\x00\x00\x00..."
1,B002,B002,B,18,Amersfort Park,Amersfort Park,"E. 38 St., E. 38 St. bet. Ave. I and Ave. J","E. 38 St., E. 38 St. bet. Ave. I and Ave. J",DPR,Neighborhood Park,3.557,100004500.0,Property,2021-04-20 21:25:30.530,b'\x8c\xf0&\xbe 2{;\xa9N\xe06\r~\x901L\xf5\x1c...,False,b'\xd7\x08\x00\x00\x01\x04\x05\x00\x00\x00\x00...
2,B003,B003,B,19,Bartel-Pritchard Square,Bartel-Pritchard Square,"Prospect Park W., 15 St.","Prospect Park W., 15 St.",DPR,Triangle/Plaza,1.711,100003839.0,Property,2021-04-20 21:25:30.530,b'\xb3m\xd3\x9f\xcb\x01\xfd\xd94\x88\x0c\xeap\...,False,b'\xd7\x08\x00\x00\x01\x04\xb9\x00\x00\x00\x00...
3,B006,B006,B,03,Beattie Square,Beattie Square,"Broadway, Stuyvesant Ave., Vernon Ave.","Broadway, Stuyvesant Ave., Vernon Ave.",DPR,Triangle/Plaza,0.012,100003773.0,Property,2021-04-20 21:25:30.530,"b'o\x98\x94\xf2\xfdH\xd36j\t/|\x0e\\?,\xb3\x87...",False,b'\xd7\x08\x00\x00\x01\x04\x04\x00\x00\x00\x80...
4,B007,B007,B,11,Bensonhurst Park,Bensonhurst Park,Cropsey Ave. bet. 21 Ave. and Bay Pkwy.,Cropsey Ave. bet. 21 Ave. and Bay Pkwy.,DPR,Recreation Field/Courts,17.500,100004816.0,Property,2021-04-20 21:25:30.530,b'\x06\xc6YC3\xa8\xa6f\x088\xaf(u\x87F\x9a\xe2...,False,b'\xd7\x08\x00\x00\x01\x04H\x00\x00\x00\x80\x9...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6431,XZ94,XZ94,X,04,GREENSTREET,GREENSTREET,East 153rd St. Bet. Concourse Village We,East 153rd St. Bet. Concourse Village We,DOT,None,0.034,100006143.0,Greenstreet,2021-04-20 21:25:30.530,b'\x89\xc7\xc1\x03\x15r\xa2\xd9\xa7B\xb4\x10\x...,False,b'\xd7\x08\x00\x00\x01\x04\xc6\x00\x00\x00\x00...
6432,XZ95,XZ95,X,04,GREENSTREET,GREENSTREET,Grand Concourse And Tudor Pl.,Grand Concourse And Tudor Pl.,DOT,None,0.002,100007214.0,Greenstreet,2021-04-20 21:25:30.530,b'<\x9a\xec\xf1\xba\x02\x8a\x92G\xdck\x930\xb2...,False,b'\xd7\x08\x00\x00\x01\x04\x05\x00\x00\x00\x80...
6433,XZ96,XZ96,X,04,GREENSTREET,GREENSTREET,Tudor Pl. And Grand Concourse,Tudor Pl. And Grand Concourse,DOT,None,0.002,100007213.0,Greenstreet,2021-04-20 21:25:30.530,b'\xc34m\xde\x8e\xab\xd2\xc7M\x1f\x93\xaf\xf3\...,False,b'\xd7\x08\x00\x00\x01\x04\x05\x00\x00\x00\x00...
6434,XZ97,XZ97,X,04,GREENSTREET,GREENSTREET,Mc Clellan St. And Grand Concourse,Mc Clellan St. And Grand Concourse,DOT,None,0.002,100006301.0,Greenstreet,2021-04-20 21:25:30.530,b'Rl\xda\x8a<H\xfa\x1bH\xefk\xc8\xe5\xfa\xec\x...,False,b'\xd7\x08\x00\x00\x01\x04\x05\x00\x00\x00\x80...


## Perform DML Steps

### Delete Records

In [13]:
#Define the common where clause
where = str(r"where objectid like '%3%'")

In [21]:
#Should be able to filter original DFs

In [15]:
#Loop through the GIS datasets, drop the _evw and perform the deletes
sql = ['delete from parksgis.dpr.{} {}'.format(g.replace('_evw', ''), where) for g in gis_tables]

In [16]:
gis_con = gis_engine.connect()

In [17]:
sql

["delete from parksgis.dpr.property where objectid like '%3%'",
 "delete from parksgis.dpr.playground where objectid like '%3%'",
 "delete from parksgis.dpr.zone where objectid like '%3%'",
 "delete from parksgis.dpr.unmapped_gisallsites where objectid like '%3%'",
 "delete from parksgis.dpr.schoolyard_to_playground where objectid like '%3%'",
 "delete from parksgis.dpr.greenstreet where objectid like '%3%'",
 "delete from parksgis.dpr.golfcourse where objectid like '%3%'",
 "delete from parksgis.dpr.restrictivedeclarationsite where objectid like '%3%'"]

In [18]:
for q in sql:
    gis_con.execute(q)

In [ ]:
#Create a list of dataframes with the original data
gis_df_post_delete = [pd.read_sql(con = gis_engine, sql = q) for q in pre_delete_sql]

### Update Records

In [ ]:
#Define the common where clause
where = str(r"where objectid like '%4%'")

In [ ]:
set_value = str(r"")

In [ ]:
#Loop through the GIS datasets, drop the _evw and perform the deletes
sql = ['update parksgis.dpr.{} set  {}'.format(g.replace('_evw', ''), where) for g in gis_tables]

### Insert Non-Duplicate Records

In [39]:
import random

In [66]:
#Define the range for generating records
n_gen = range(0, 100)

In [37]:
#Define the borough portion of IDs
boro = ['B','Q','M','R','X'] 

In [68]:
#Combine the borough portion with the formated (padded with leading 0) random numbers
nums = [boro[random.randint(0,4)] + f'{random.randint(0, 999):03}' for i in n_gen]

In [69]:
nums

['B031',
 'Q987',
 'B623',
 'M749',
 'X520',
 'M462',
 'M809',
 'Q353',
 'Q165',
 'B370',
 'R905',
 'X598',
 'X685',
 'Q481',
 'B949',
 'B851',
 'M019',
 'M477',
 'Q699',
 'X483',
 'X843',
 'M170',
 'M663',
 'B356',
 'M900',
 'R527',
 'R218',
 'Q946',
 'B983',
 'R354',
 'R197',
 'Q556',
 'X008',
 'M820',
 'B829',
 'M805',
 'R978',
 'B089',
 'R160',
 'R319',
 'X106',
 'Q021',
 'M430',
 'Q593',
 'X800',
 'R853',
 'B223',
 'B681',
 'R181',
 'M995',
 'M275',
 'Q573',
 'B552',
 'M260',
 'X161',
 'B780',
 'M970',
 'Q512',
 'R311',
 'Q163',
 'X018',
 'M140',
 'X586',
 'M959',
 'R113',
 'Q065',
 'M686',
 'B193',
 'X514',
 'X540',
 'B129',
 'X165',
 'X787',
 'M418',
 'X780',
 'M050',
 'Q447',
 'M505',
 'M495',
 'Q822',
 'X826',
 'Q875',
 'B962',
 'M636',
 'X842',
 'B029',
 'R082',
 'R716',
 'B802',
 'R813',
 'B536',
 'X160',
 'B802',
 'M985',
 'Q449',
 'M960',
 'M505',
 'Q589',
 'Q327',
 'B634']

### Insert Duplicate Records